<a href="https://colab.research.google.com/github/RahulDogra-92/Deep-Learning-using-Pytorch/blob/main/Convolutional_Neural_Networks.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#Convolutions in Pytorch

In [1]:
import torch
import torch.nn.functional as F

In [2]:
image = torch.rand(16,3,32,32)
filter = torch.rand(1,3,5,5)
out_feat_F = F.conv2d(image, filter, stride=1, padding=0)

In [3]:
print(out_feat_F.shape)

torch.Size([16, 1, 28, 28])


#Convolution operator - Functional way
how to build convolutional layers using functional package.

In [4]:
# Create 10 random images
image = torch.rand(10, 1, 28, 28)

# Create 6 filters
filters = torch.rand(6, 1, 3, 3)

# Convolve the image with the filters
output_feature = F.conv2d(image, filters, stride=1, padding=1)
print(output_feature.shape)

torch.Size([10, 6, 28, 28])


#Max-pooling in Pytorch

In [5]:
import torch
import torch.nn.functional as F

In [6]:
im = torch.Tensor([[[[3, 1, 3, 5], [6, 0, 7, 9],
                     [3, 2, 1, 4], [0, 2, 4, 3]]]])

output_feature_F = F.max_pool2d(im, 2)
print(output_feature_F)

tensor([[[[6., 9.],
          [3., 4.]]]])


/usr/local/lib/python3.7/dist-packages/torch/nn/functional.py:718: UserWarning: Named tensors and all their associated APIs are an experimental feature and subject to change. Please do not use them for anything important until they are released as stable. (Triggered internally at  /pytorch/c10/core/TensorImpl.h:1156.)
  return torch.max_pool2d(input, kernel_size, stride, padding, dilation, ceil_mode)


In [7]:
im = torch.Tensor([[[[3, 1, 3, 5], [6, 0, 7, 9],
                     [3, 2, 1, 4], [0, 2, 4, 3]]]])

output_feature_F = F.avg_pool2d(im, 2)
print(output_feature_F)

tensor([[[[2.5000, 6.0000],
          [1.7500, 3.0000]]]])


#Max-pooling operator

In [8]:
# Build a pooling operator with size `2`.
max_pooling = torch.nn.MaxPool2d(2)

# Apply the pooling operator
output_feature = max_pooling(im)

# Use pooling operator in the image
output_feature_F = F.max_pool2d(im, 2)

# print the results of both cases
print(output_feature)
print(output_feature_F)

tensor([[[[6., 9.],
          [3., 4.]]]])
tensor([[[[6., 9.],
          [3., 4.]]]])


#Average-pooling operator

In [9]:
# Build a pooling operator with size `2`.
avg_pooling = torch.nn.AvgPool2d(2)

# Apply the pooling operator
output_feature = avg_pooling(im)

# Use pooling operator in the image
output_feature_F = F.avg_pool2d(im, 2)

# print the results of both cases
print(output_feature)
print(output_feature_F)

tensor([[[[2.5000, 6.0000],
          [1.7500, 3.0000]]]])
tensor([[[[2.5000, 6.0000],
          [1.7500, 3.0000]]]])


#Convolutional Neural Networks

#Imports

In [15]:
import torch
import torchvision
import torch.nn as nn
import torchvision.transforms as transforms
import torch.nn.functional as F
import torch.optim as optim

#Dataloaders

In [16]:
transform = transforms.Compose(
    [transforms.ToTensor(),
     transforms.Normalize((0.4914, 0.48216, 0.44543),
                          (0.24703, 0.24349, 0.26519))])

trainset = torchvision.datasets.CIFAR10(root="./data", train = True, download = True, transform = transform)
testset = torchvision.datasets.CIFAR10(root="./data", train = False, download = True, transform = transform)


trainloader = torch.utils.data.DataLoader(trainset, batch_size=32, shuffle = True, num_workers = 4)
testloader = torch.utils.data.DataLoader(testset, batch_size=32, shuffle = False, num_workers = 4)

Files already downloaded and verified
Files already downloaded and verified


/usr/local/lib/python3.7/dist-packages/torch/utils/data/dataloader.py:481: UserWarning: This DataLoader will create 4 worker processes in total. Our suggested max number of worker in current system is 2, which is smaller than what this DataLoader is going to create. Please be aware that excessive worker creation might get DataLoader running slow or even freeze, lower the worker number to avoid potential slowness/freeze if necessary.
  cpuset_checked))


#Building a CNN

In [23]:
class Net(nn.Module):
  def __init__(self, num_classes=10):
    super(Net, self).__init__()
    self.conv1 = nn.Conv2d(in_channels=3, out_channels=32, kernel_size=3, padding=1)
    self.conv2 = nn.Conv2d(in_channels=32, out_channels=64, kernel_size=3, padding=1)
    self.conv3 = nn.Conv2d(in_channels=64, out_channels=128, kernel_size=3, padding=1)
    self.pool = nn.MaxPool2d(2, 2)
    self.fc = nn.Linear(128 * 4 * 4, num_classes)

  def forward(self, x):

    x = self.pool(F.relu(self.conv1(x)))
    x = self.pool(F.relu(self.conv2(x)))
    x = self.pool(F.relu(self.conv3(x)))
    x = x.view(-1, 128 * 4 * 4)
    return self.fc(x)

#Optimizer and Loss Function

In [24]:
net = Net()
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(net.parameters(), lr=3e-4)

#Training a CNN

In [ ]:
for epoch in range(10):
  for i, data in enumerate(trainloader, 0):

    #get the inputs
    inputs, labels = data

    #Zero the parameters the gradients
    optimizer.zero_grad()

    #Forward + backward + optimize
    outputs = net(inputs)
    loss = criterion(outputs, labels)
    loss.backward()      #Compute the weights using backward()
    optimizer.step()     #Update the weights using optimizer

print('Finised Training')

/usr/local/lib/python3.7/dist-packages/torch/utils/data/dataloader.py:481: UserWarning: This DataLoader will create 4 worker processes in total. Our suggested max number of worker in current system is 2, which is smaller than what this DataLoader is going to create. Please be aware that excessive worker creation might get DataLoader running slow or even freeze, lower the worker number to avoid potential slowness/freeze if necessary.
  cpuset_checked))


#Evaluating the results

In [ ]:
correct, total = 0,0
predictions = []
net.eval()

for i, data in enumerate(testloader, 0):
  inputs, labels = data
  outputs = net(inputs)
  _, predicted = torch.max(outputs.data, 1)
  predictions.append(outputs)
  total += label.size(0)
  correct += (predicted ==labels).sum().item()

print('The testing set accuracy of the network is: %d %%' % (
    100 * correct / total))





